In [ ]:
import sys


sys.path.append("..")


In [ ]:
import constants
import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


In [ ]:
import datahandler
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd 
import folium

dataset_id = "oslo"

data_preprocessor = datahandler.DataPreprocessorOUS()
data_preprocessor.execute()

data_loader = datahandler.DataLoader()
data_loader.execute()

df = data_loader.enhanced_incidents_df


In [ ]:
# Step 1: Load the GeoJSON files into GeoDataFrames
akershus_gdf = gpd.read_file(os.path.join(constants.PROJECT_DIRECTORY_PATH, "data", "ssb_2019_akershus_polygon_epsg4326.geojson"))
oslo_gdf = gpd.read_file(os.path.join(constants.PROJECT_DIRECTORY_PATH, "data", "ssb_2019_oslo_polygon_epsg4326.geojson"))
urban_settlements_gdf = gpd.read_file(os.path.join(constants.PROJECT_DIRECTORY_PATH, "data", "ssb_2021_urban_settlements_polygon_epsg4326.geojson"))

# Step 2: Combine Akershus and Oslo GeoDataFrames
combined_gdf = gpd.GeoDataFrame(pd.concat([akershus_gdf, oslo_gdf], ignore_index=True))

# Step 3: Perform the spatial join
urban_clipped_akershus_oslo = gpd.overlay(urban_settlements_gdf, combined_gdf, how='intersection')

In [ ]:
import seaborn as sns

m = folium.Map(location=[59.9139, 10.7522], zoom_start=10)  # Coordinates for Oslo

akershus_gdf['dissolve_field'] = 'Region'
oslo_gdf['dissolve_field'] = 'Region'
combined_gdf = pd.concat([akershus_gdf, oslo_gdf], ignore_index=True)
dissolved_gdf = combined_gdf.dissolve(by='dissolve_field')

oslo_akershus_geojson = dissolved_gdf.to_json()
urban_clipped_geojson = urban_clipped_akershus_oslo.to_json()

palette = sns.color_palette("husl", 4)  # "husl" is just an example, and 2 is the number of colors you want
region_color = palette.as_hex()[1]
urban_color = palette.as_hex()[0]

folium.GeoJson(
    oslo_akershus_geojson,
    name='Oslo og Akershus',
    style_function=lambda x: {'fillColor': region_color, 'color': region_color}
).add_to(m)

folium.GeoJson(
    urban_clipped_geojson,
    name='Urban Settlements Clipped',
    style_function=lambda x: {'fillColor': urban_color, 'color': urban_color}
).add_to(m)

folium.LayerControl().add_to(m)

m


In [ ]:
CODE HERE